<a href="https://colab.research.google.com/github/carteras/IT-CBR/blob/main/colab_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
!pip install names

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 789.1/789.1 kB 17.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for names: filename=names-0.3.0-py3-none-any.whl size=803682 sha256=29ff405e09a1bdddc618881a99dfdeb67fe785c97272ff885de4064d90564ca3
  Stored in directory: /root/.cache/pip/wheels/fc/9a/6f/78f4282bbcaa2d8c678b73c54c0bb1b7a04009f0d7cec79fce
Successfully built names


In [10]:

from google.colab import auth
from google.colab import drive
import pandas as pd


In [8]:
auth.authenticate_user()
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


You probably want to change these locations. Just type them in the text field to the right. No need to play with variable names here.

In [20]:
data_address = "/content/drive/My Drive/flipped/teaching.and.learning/colab"#@param {type: "string"}
data = "data.csv" #@param {type: "string"}

Let's make some data!

In [54]:
from dataclasses import dataclass, field
from random import choice

@dataclass
class Student:
  """class for keeping track of students"""
  given_name: str
  family_name: str
  subject_name: str
  ai_scores: list[float] = field(default_factory=list)

  def randomly_make_grades(self, ai_number = 4):
    from random import random
    for i in range(ai_number):
      self.ai_scores.append(round(random()*100, 2))

  def __repr__(self):
    return "\n".join(
        [f"{self.given_name},{self.family_name},{self.subject_name},ai{i+1},{score}"
         for i, score in enumerate(self.ai_scores)])


In [55]:
def make_students(number_of_students:int = 125) -> list[Student] :
  """constructs a list of random students"""
  import names
  out = []
  for _ in range(number_of_students):
    s = Student(
      given_name = names.get_first_name(),
      family_name = names.get_last_name(),
      subject_name = choice(subjects),
    )
    s.randomly_make_grades()
    out.append(s)
  return out

In [56]:
subjects = ['networking and security', 'digital technologies', 'engineering']
students = make_students(125)
with open(f"{data_address}/{data}", 'w') as fd:
  fd.write(f"given_name,family_name,subject,ai,raw_score\n")
  for student in students:
    fd.write(f"{student}\n")

Let's use it with pandas

In [57]:
df = pd.read_csv(f"{data_address}/{data}")

In [58]:
df.head()

,given_name,family_name,subject,ai,raw_score
0,Leon,Hoover,digital technologies,ai1,2.18
1,Leon,Hoover,digital technologies,ai2,7.40
2,Leon,Hoover,digital technologies,ai3,67.66
3,Leon,Hoover,digital technologies,ai4,6.60
4,Derrick,Cooper,networking and security,ai1,19.91


In [59]:
ai1_df = df[df.ai == "ai1"]
ai1_df.head()

,given_name,family_name,subject,ai,raw_score
0,Leon,Hoover,digital technologies,ai1,2.18
4,Derrick,Cooper,networking and security,ai1,19.91
8,Rachel,Markow,networking and security,ai1,6.06
12,Dawn,Bodden,engineering,ai1,77.23
16,Jose,Lankford,engineering,ai1,22.88


In [60]:
random_subject = df.subject.sample().values[0]
random_subject

'networking and security'

In [61]:
random_subject_ai1_df = df[(df.ai == "ai1") & (df.subject == random_subject)]
random_subject_ai1_df.head()

,given_name,family_name,subject,ai,raw_score
4,Derrick,Cooper,networking and security,ai1,19.91
8,Rachel,Markow,networking and security,ai1,6.06
20,Owen,Mosher,networking and security,ai1,12.20
44,Richard,Spinner,networking and security,ai1,76.43
56,Beatrice,Downing,networking and security,ai1,53.56


In [62]:
from scipy.stats import zscore
random_subject_ai1_df['zscore'] = zscore(df.raw_score)

<ipython-input-62-a1af5b688d92>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  random_subject_ai1_df['zscore'] = zscore(df.raw_score)


In [63]:
random_subject_ai1_df.head()

,given_name,family_name,subject,ai,raw_score,zscore
4,Derrick,Cooper,networking and security,ai1,19.91,-1.011212
8,Rachel,Markow,networking and security,ai1,6.06,-1.503123
20,Owen,Mosher,networking and security,ai1,12.20,-1.285048
44,Richard,Spinner,networking and security,ai1,76.43,0.996212
56,Beatrice,Downing,networking and security,ai1,53.56,0.183937


In [64]:
random_subject_ai1_df.raw_score.mean()

46.904